In [ ]:
pip install nltk

In [ ]:
#!/usr/bin/env python
# coding: utf-8

#  ##                                             Fake Job Posting Prediction

# ## 1) Importing Library 
# I started yo import all the libraries which I need. In there, I import some models library(MultinominalNB and LinearSVC)and some preprocessing library for the data cleaning.

# In[158]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  
from collections import defaultdict
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk import pos_tag

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
from wordcloud import WordCloud, STOPWORDS



from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 
import scikitplot as skplt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


# ### 2) Importing the dataset and describing 

# In[171]:


data = pd.read_csv("../input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")


# In[172]:


data.head()


# In[173]:


data.info()


# In[174]:


data.shape


# In[175]:


data.describe()


# In[176]:


sns.countplot(data.employment_type)


# ## 3) Preprocessing step 
# In this step, I start to look at my dataset to fill the NA data to space. After that, I keep going on with merge the all columns into a column which is all_text. So, I can clean the dataset. 

# In[177]:


data.isna().sum()


# In[178]:


data.fillna(" ",inplace = True)


# In[179]:


data["all_text"] = data["title"] + " " + data["location"] + " " + data["department"] + " " + data["company_profile"] + " " + data["description"] + " " + data["requirements"] + " " + data["benefits"] + " " + data["employment_type"] + " " + data["required_experience"] + " " + data["required_education"] + " " + data["industry"] + " " + data["function"]


# In[180]:


data


# I consider the text as a sequence of words can be a reasonable choice. And, I used to find boundaries of words by splitting sentences by space of punctuation. So, I am going to convert text to lowercase. After that, I keep going with Lemmatization. It is the process of grouping together the inflected forms of a word so they can be analyzed as a single item, identified by the word’s lemma, or dictionary form.

# In[181]:


data['all_text'] = [entry.lower() for entry in data['all_text']]


# In[182]:


from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
print(data['all_text'])
data['last_text'] = data['all_text'].apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split() ]))
print(data['last_text'])


# In the below codes, I consider with removal stop words. These are the most common words in a language like 'some', 'more', 'its'. When I remove all of the stop words the sentence becomes more cleaner.

# In[183]:


stop_words = set(stopwords.words('english'))


# In[184]:


print(stop_words)


# In[185]:


data['last_text'] = data['last_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(data['last_text'])


# Now we can see the result of the preparations we made here in the preliminary preparation section, here we can find the most written words in the actual job postings in the fraudulent column on the dataset. I use to plot using the word cloud.

# In[186]:



plt.figure(figsize = (20,20)) # Text that is not fraudulent(0)
wc = WordCloud(width = 1600 , height = 800 , max_words = 3000).generate(" ".join(data[data.fraudulent == 0].last_text))
plt.imshow(wc , interpolation = 'bilinear')


# In the below image, we are going to see which keyword groups consist of fake job postings.

# In[187]:


plt.figure(figsize = (20,20)) # Text that is not fraudulent(0)
wc = WordCloud(width = 1600 , height = 800 , max_words = 3000).generate(" ".join(data[data.fraudulent == 1].last_text))
plt.imshow(wc , interpolation = 'bilinear')


# ## 4) Transforming anf Vectorizing the all_text
# 
# After the preprocessing part, I need to have test and training data all set up. So, I start by splitting data and use a test size of 0.30. Now, I can create vectorized representations of the all_text. 

# In[188]:


y = data["fraudulent"]


# In[189]:


X_train, X_test, y_train, y_test = train_test_split(data["last_text"], y, random_state=0, test_size=.30)


# In[190]:


count_vectorizer = CountVectorizer(stop_words="english", min_df=0.05, max_df=0.9)

# Create count train and test variables
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

# Initialize tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words="english", min_df=0.05, max_df=0.9)

# Create tfidf train and test variables
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)


# In[191]:


print(count_train.shape)
print(count_test.shape)
print(tfidf_train.shape)
print(tfidf_test.shape)


# ### 5) Modelling with training a multinomial naive bayes model
# 
# So, I have vectorized data. I can train the multinomial naive Bayes model. And, we are going to see the accuracy results. The accuracy score is %96 which is a very good score for the naive classifier.

# In[192]:



# Create a MulitnomialNB model
tfidf_nb = MultinomialNB()
tfidf_nb.fit(tfidf_train, y_train)
# ... Train your model here ...

# Run predict on your TF-IDF test data to get your predictions
tfidf_nb_pred = tfidf_nb.predict(tfidf_test)

# Calculate the accuracy of your predictions
tfidf_nb_score = metrics.accuracy_score(y_test, tfidf_nb_pred)

# Create a MulitnomialNB model
count_nb = MultinomialNB()
# ... Train your model here ...
count_nb.fit(count_train, y_train)
# Run predict on your count test data to get your predictions
count_nb_pred = count_nb.predict(count_test)

# Calculate the accuracy of your predictions
count_nb_score = metrics.accuracy_score(y_test, count_nb_pred)
tf_nb = confusion_matrix(y_test, tfidf_nb_pred)
c_nb = confusion_matrix(y_test, count_nb_pred)

print('NaiveBayes Tfidf Score: ', tfidf_nb_score)
print('NaiveBayes Count Score: ', count_nb_score)


# I try out another classifier which is Linear svc. 

# In[118]:



# Create a LinearSVM model
tfidf_svc = LinearSVC()

# ... Train your model here ...
tfidf_svc.fit(tfidf_train, y_train)
# Run predict on your tfidf test data to get your predictions
tfidf_svc_pred = tfidf_svc.predict(tfidf_test)

# Calculate your accuracy using the metrics module
tfidf_svc_score = metrics.accuracy_score(y_test, tfidf_svc_pred)

print("LinearSVC Score:   %0.3f" % tfidf_svc_score)

# Calculate the confusion matrices for the tfidf_svc model

svc_cm = confusion_matrix(y_test, tfidf_svc_pred)

    

ax= plt.subplot()
sns.heatmap(svc_cm, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['real', 'fake']); ax.yaxis.set_ticklabels(['real', 'fake']);


# Predicting the sentiment for the documents in our test set I can use the predicting method of the RandomForestClassifier class as shown below and I got a very high accuracy score.

# In[193]:



rf = RandomForestClassifier(bootstrap=True)

rf.fit(tfidf_train, y_train)

# Cross validation of 5 folds
score = cross_val_score(rf, tfidf_train, y_train)

print(f'Prediction score: {np.mean(score) * 100:.2f}%')


# ### 6) Evaluating my model using a confusiın matrix
# The confusion matrix gives additional insight into accuracy by class and intuition for precision and recalls efficiency. And, we are going to see them that is so high efficiency. From the output, it can be seen that our model achieved an accuracy of 97.3%, which is very good given the fact that we randomly chose all the parameters for CountVectorizer as well as for our random forest algorithm.

# In[194]:


from sklearn.metrics import classification_report
print(classification_report(y_test, tfidf_nb_pred, target_names = ['0','1']))


# In[195]:


print(classification_report(y_test, tfidf_svc_pred, target_names = ['0','1']))


# In[204]:


plt.figure(figsize = (4,4))
sns.heatmap(svc_cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = ['Fake','Not Fake'] , yticklabels = ['Fake','Not Fake'])


# In below the plots, we are going to benchmark other models using a confusion matrix. 

# In[205]:


plt.figure(figsize=(20,15))

plt.suptitle("Confusion Matrixes",fontsize=20)

plt.subplot(2,3,1)
plt.title("LinearSVC Confusion Matrix")
sns.heatmap(svc_cm,cbar=False,annot=True,cmap="Greens",fmt="d")

plt.subplot(2,3,2)
plt.title("MulitnomialNB TF Confusion Matrix")
sns.heatmap(tf_nb,cbar=False,annot=True,cmap="Greens",fmt="d")

plt.subplot(2,3,3)
plt.title("MulitnomialNB Count Confusion Matrix")
sns.heatmap(c_nb,cbar=False,annot=True,cmap="Greens",fmt="d")


plt.show()


# IN ADDITION: All_text full data visualization

# In[148]:


text = ''
for news in data.all_text.values:
    text += f" {news}"
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()
del text


# ### 7) Conclusion
# 
# At the beginning of the project, I start with importing and cleaning data. After that, I continue with preprocessing by doing text cleaning. So, I prepare my dataset to build a model. I used my model into a fake job dataset. At the end of the project, I compare and evaluate the models using a confusion matrix. The results, the highest accuracy score is LinearSVC model. And, when we looked at the confusion matrix scores, true positive is so high to others in every model, and false negative and false positive is very low. So, we are going to which are real and fake. 

# ### 8) Reference
# [Kaggle notebooks](https://www.kaggle.com/madz2000/text-classification-using-keras-nb-97-accuracy) and
# [Text classification pages](https://stackabuse.com/text-classification-with-python-and-scikit-learn/)

# In[ ]: